In [1]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt
import PIL.Image as Image
import gym
import random
import math as m
from gym import Env, spaces
import time

font = cv2.FONT_HERSHEY_COMPLEX_SMALL

In [14]:
class ChopperScape(Env):
    def __init__(self):
        super(ChopperScape, self).__init__()
        

        
        # Define a 2-D observation space
        self.observation_shape = (600, 800, 3)
        self.observation_space = spaces.Box(low = np.zeros(self.observation_shape), 
                                            high = np.ones(self.observation_shape),
                                            dtype = np.float16)
    
        
        # Define an action space ranging from 0 to 4
        self.action_space = spaces.Discrete(8,)
                        
        # Create a canvas to render the environment images upon 
        self.canvas = np.ones(self.observation_shape) * 1
        
        # Define elements present inside the environment
        self.elements = []
        
        # Maximum fuel chopper can take at once
        self.max_fuel = 1000
        self.miss=0

        # Permissible area of helicper to be 
#         self.y_min = int (self.observation_shape[0] * 0.1)
#         self.x_min = 0
#         self.y_max = int (self.observation_shape[0] * 0.9)
#         self.x_max = self.observation_shape[1]
        self.y_min = 0
        self.x_min = 0
        self.y_max = self.observation_shape[0]
        self.x_max = self.observation_shape[1]
        
        # velocity angle projectile
        self.v=1
        self.an=0
        self.l1=[]
        self.dx1=0
        
    def draw_elements_on_canvas(self):
        # Init the canvas 
        self.canvas = np.ones(self.observation_shape) * 1

        # Draw the heliopter on canvas
        for elem in self.elements:
            elem_shape = elem.icon.shape
            x,y = elem.x, elem.y
            #print(x,y,elem_shape[0],elem_shape[1])
            self.canvas[y : y + elem_shape[1], x:x + elem_shape[0]] = elem.icon

        text = 'Hit Reward: {} | Miss: {} | Rewards: {}'.format(self.hr,self.miss,self.ep_return)

        # Put the info on canvas 
        self.canvas = cv2.putText(self.canvas, text, (10,20), font,  
                   0.8, (0,0,0), 1, cv2.LINE_AA)
        #initialize variables
        #velocity, gravity
        g = 9.8
        #increment theta 25 to 60 then find  t, x, y
        #define x and y as arrays
        v=self.v
        theta=self.an
        a=0
        #theta = np.arange(m.pi/6, m.pi/3, m.pi/36)
        R=v**2*m.sin(2*theta)/g
        self.dx1=R
        h=v**2*m.sin(theta)**2/(2*g)
        t = np.linspace(0, R/(v*m.cos(theta)), num=100) # Set time as 'continous' parameter.
        #for i in theta: # Calculate trajectory for every angle
        x1 = []
        y1 = []
        l=[]
        mh=0
        for k in t:
            x = ((v*k)*np.cos(theta)) # get positions at every point in time
            y = ((v*k)*np.sin(theta))-((0.5*g)*(k**2))
            if(y>=0):
                x1.append(x)
                y1.append(y)
                if(mh<y):
                    mh=y
                #l.append([x*m.cos(a)+y*m.sin(a),-x*m.sin(a)+y*m.cos(a)])
        for i in range(len(x1)):
            l.append([x1[i],(mh-y1[i])+600-h])
        self.l1=l
        pts = np.array(l,np.int32)
        pts = pts.reshape((-1, 1, 2))

        isClosed = False

        # Green color in BGR
        color = (0, 255, 255)

        # Line thickness of 8 px
        thickness = 3

        # Using cv2.polylines() method
        # Draw a Green polygon with
        # thickness of 1 px
        self.canvas = cv2.polylines(self.canvas, [pts],
                              isClosed, color,
                              thickness)

        
        
    def reset(self):
        # Reset the fuel consumed
        self.fuel_left = self.max_fuel

        # Reset the reward
        self.ep_return  = 0
        self.hr=0

        # Number of birds
        self.bird_count = 0
        self.fuel_count = 0
        self.target_count=0

        # Determine a place to intialise the chopper in
#         x = random.randrange(int(self.observation_shape[0] * 0.05), int(self.observation_shape[0] * 0.10))
#         y = random.randrange(int(self.observation_shape[1] * 0.15), int(self.observation_shape[1] * 0.20))
        x=50
        y=50
        
        # Intialise the chopper
        self.chopper = Chopper("chopper", self.x_max, self.x_min, self.y_max, self.y_min)
        self.chopper.set_position(x,y)

        # Intialise the elements 
        self.elements = [self.chopper]

        # Reset the Canvas 
        self.canvas = np.ones(self.observation_shape) * 1

        # Draw elements on the canvas
        self.draw_elements_on_canvas()


        # return the observation
        return self.canvas

    def render(self, mode = "human"):
        assert mode in ["human", "rgb_array"], "Invalid mode, must be either \"human\" or \"rgb_array\""
        if mode == "human":
            cv2.imshow("Game", self.canvas)
            cv2.waitKey(10)

        elif mode == "rgb_array":
            return self.canvas

    def close(self):
        cv2.destroyAllWindows()

    def get_action_meanings(self):
        return {0: "Shoot", 1: "Do Nothing"}

    def has_collided(self, elem1, elem2):
        x_col = False
        y_col = False

        elem1_x, elem1_y = elem1.get_position()
        elem2_x, elem2_y = elem2.get_position()

        if 2 * abs(elem1_x - elem2_x) <= (elem1.icon_w + elem2.icon_w):
            x_col = True

        if 2 * abs(elem1_y - elem2_y) <= (elem1.icon_h + elem2.icon_h):
            y_col = True

        if x_col and y_col:
            return True

        return False

    def target_hit(self, elem1):

        elem1_x, elem1_y = elem1.get_position()
        x=self.dx1
        half=elem1.icon_w//2
        th=elem1_x+half
        r=0

        if(x>=elem1_x and x<=elem1_x+elem1.icon_w):
            if(x>th):
                r=((elem1_x+elem1.icon_w-x)/half)*100
            else:
                r=((x-elem1_x)/half)*100
            print('Lastx: {} | Tarx: {} | Hit Reward: {}'.format(x,elem1_x,r))
            self.miss=0
        else:
            r=-10
            self.miss+=1
        
        

        return r
    
    def rectify(self):
        if(self.v<=0):
            self.v=1
        elif(self.v>=80):
            self.v=79
        if(self.an<=0):
            self.an=0
        elif(self.an>=(90)*(m.pi/180)):
            self.an=(90)*(m.pi/180)
        return 


    
    def step(self, action):
        # Flag that marks the termination of an episode
        done = False

        # Assert that it is a valid action 
        assert self.action_space.contains(action), "Invalid Action"

        # Decrease the fuel counter 
        self.fuel_left -= 1 

        # Reward for executing a step.
        reward = 1
        dx=0
        dy=0
        k=1
        g=9.8
        # apply the action to the chopper
        if action == 0: 
            self.v+=1
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 1:
            self.an+=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        if action == 2: 
            self.v-=1
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 3:
            self.an-=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 4:
            self.v+=1
            self.an+=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 5:
            self.v-=1
            self.an-=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 6:
            self.v+=1
            self.an-=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 7:
            self.v-=1
            self.an+=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))

        # Spawn a bird at the right edge with prob 0.01
        if random.random() < 0.0:

            # Spawn a bird
            spawned_bird = Bird("bird_{}".format(self.bird_count), self.x_max, self.x_min, self.y_max, self.y_min)
            self.bird_count += 1

            # Compute the x,y co-ordinates of the position from where the bird has to be spawned
            # Horizontally, the position is on the right edge and vertically, the height is randomly 
            # sampled from the set of permissible values
            bird_x = self.x_max 
            bird_y = random.randrange(self.y_min, self.y_max)
            spawned_bird.set_position(self.x_max, bird_y)

            # Append the spawned bird to the elements currently present in Env. 
            self.elements.append(spawned_bird)    

        # Spawn a fuel at the bottom edge with prob 0.01
        if random.random() < 0.0:
            # Spawn a fuel tank
            spawned_fuel = Fuel("fuel_{}".format(self.fuel_count), self.x_max, self.x_min, self.y_max, self.y_min)
            self.fuel_count += 1

            # Compute the x,y co-ordinates of the position from where the fuel tank has to be spawned
            # Horizontally, the position is randomly chosen from the list of permissible values and 
            # vertically, the position is on the bottom edge
            fuel_x = random.randrange(self.x_min, self.x_max)
            fuel_y = self.y_max
            spawned_fuel.set_position(fuel_x, fuel_y)

            # Append the spawned fuel tank to the elemetns currently present in the Env.
            self.elements.append(spawned_fuel)   
            
        # Spawn a target at the bottom edge with prob 0.01
        if random.random() < 0.1:
            # Spawn a fuel tank
            spawned_target = Target("target_{}".format(self.target_count), self.x_max, 0, 600, 0)
            self.target_count += 1

            # Compute the x,y co-ordinates of the position from where the fuel tank has to be spawned
            # Horizontally, the position is randomly chosen from the list of permissible values and 
            # vertically, the position is on the bottom edge
            target_x = random.randrange(self.x_min, self.x_max)
            #target_x=200
            target_y = self.y_max
            spawned_target.set_position(target_x, target_y)

            # Append the spawned fuel tank to the elemetns currently present in the Env.
            self.elements.append(spawned_target)   


        # For elements in the Ev
        for elem in self.elements:
            if isinstance(elem, Bird):
                # If the bird has reached the left edge, remove it from the Env
                if elem.get_position()[0] <= self.x_min:
                    self.elements.remove(elem)
                else:
                    # Move the bird left by 5 pts.
                    elem.move(-5,0)

                # If the bird has collided.
                if self.has_collided(self.chopper, elem):
                    # Conclude the episode and remove the chopper from the Env.
                    done = True
                    reward = -10
                    self.elements.remove(self.chopper)

            if isinstance(elem, Fuel):
                # If the fuel tank has reached the top, remove it from the Env
                if elem.get_position()[1] <= self.y_min:
                    self.elements.remove(elem)
                else:
                    # Move the Tank up by 5 pts.
                    elem.move(0, -5)

                # If the fuel tank has collided with the chopper.
                if self.has_collided(self.chopper, elem):
                    # Remove the fuel tank from the env.
                    self.elements.remove(elem)

                    # Fill the fuel tank of the chopper to full.
                    self.fuel_left = self.max_fuel

            if isinstance(elem, Target):
                # If the fuel tank has reached the top, remove it from the Env
                if elem.get_position()[1] <= self.y_min:
                    self.elements.remove(elem)
                else:
                    # Move the Tank up by 5 pts.
                    elem.move(0,0)
        
                if self.has_collided(self.chopper, elem):
                    # Conclude the episode and remove the chopper from the Env.
                    done = True
                    reward = self.target_hit(elem)
                    self.elements.remove(self.chopper)


                # If the fuel tank has collided with the chopper.
                if (self.target_count>=2):
                    # Remove the fuel tank from the env.
                    self.elements.remove(elem)
                    self.target_count-=1
                reward=self.target_hit(elem)
                self.hr+=reward
                if(self.miss>=100):
                    done=True
                    


        # Increment the episodic return
        self.ep_return += 1

        # Draw elements on the canvas
        self.draw_elements_on_canvas()

        # If out of fuel, end the episode.
        if self.fuel_left == 0:
            done = True

        return self.canvas, reward, done, []

In [18]:
class ChopperScape(Env):
    def __init__(self):
        super(ChopperScape, self).__init__()
        

        
        # Define a 2-D observation space
        self.observation_shape = (600, 800, 3)
        self.observation_space = spaces.Box(low = np.zeros(self.observation_shape), 
                                            high = np.ones(self.observation_shape),
                                            dtype = np.float16)
    
        
        # Define an action space ranging from 0 to 7
        self.action_space = spaces.Discrete(8,)
                        
        # Create a canvas to render the environment images upon 
        self.canvas = np.ones(self.observation_shape) * 1
        
        # Define elements present inside the environment
        self.elements = []
        
        # Maximum fuel chopper can take at once
        self.max_fuel = 1000
        self.miss=0

        # Permissible area of helicper to be 
#         self.y_min = int (self.observation_shape[0] * 0.1)
#         self.x_min = 0
#         self.y_max = int (self.observation_shape[0] * 0.9)
#         self.x_max = self.observation_shape[1]
        self.y_min = 0
        self.x_min = 0
        self.y_max = self.observation_shape[0]
        self.x_max = self.observation_shape[1]
        
        # velocity angle projectile
        self.v=1
        self.an=0
        self.l1=[]
        self.dx1=0
        
    def draw_elements_on_canvas(self):
        # Init the canvas 
        self.canvas = np.ones(self.observation_shape) * 1

        # Draw the heliopter on canvas
        for elem in self.elements:
            elem_shape = elem.icon.shape
            x,y = elem.x, elem.y
            #print(x,y,elem_shape[0],elem_shape[1])
            self.canvas[y : y + elem_shape[1], x:x + elem_shape[0]] = elem.icon

        text = 'Hit Reward: {} | Miss: {} | Rewards: {}'.format(self.hr,self.miss,self.ep_return)

        # Put the info on canvas 
        self.canvas = cv2.putText(self.canvas, text, (10,20), font,  
                   0.8, (0,0,0), 1, cv2.LINE_AA)
        #initialize variables
        #velocity, gravity
        g = 9.8
        #increment theta 25 to 60 then find  t, x, y
        #define x and y as arrays
        v=self.v
        theta=self.an
        a=0
        #theta = np.arange(m.pi/6, m.pi/3, m.pi/36)
        R=v**2*m.sin(2*theta)/g
        self.dx1=R
        h=v**2*m.sin(theta)**2/(2*g)
        t = np.linspace(0, R/(v*m.cos(theta)), num=100) # Set time as 'continous' parameter.
        #for i in theta: # Calculate trajectory for every angle
        x1 = []
        y1 = []
        l=[]
        mh=0
        for k in t:
            x = ((v*k)*np.cos(theta)) # get positions at every point in time
            y = ((v*k)*np.sin(theta))-((0.5*g)*(k**2))
            if(y>=0):
                x1.append(x)
                y1.append(y)
                if(mh<y):
                    mh=y
                #l.append([x*m.cos(a)+y*m.sin(a),-x*m.sin(a)+y*m.cos(a)])
        for i in range(len(x1)):
            l.append([x1[i],(mh-y1[i])+600-h])
        self.l1=l
        pts = np.array(l,np.int32)
        pts = pts.reshape((-1, 1, 2))

        isClosed = False

        # Green color in BGR
        color = (0, 255, 255)

        # Line thickness of 8 px
        thickness = 3

        # Using cv2.polylines() method
        # Draw a Green polygon with
        # thickness of 1 px
        self.canvas = cv2.polylines(self.canvas, [pts],
                              isClosed, color,
                              thickness)

        
        
    def reset(self):
        # Reset the fuel consumed
        self.fuel_left = self.max_fuel

        # Reset the reward
        self.ep_return  = 0
        self.hr=0

        # Number of birds
        self.bird_count = 0
        self.fuel_count = 0
        self.target_count=0

        # Determine a place to intialise the chopper in
#         x = random.randrange(int(self.observation_shape[0] * 0.05), int(self.observation_shape[0] * 0.10))
#         y = random.randrange(int(self.observation_shape[1] * 0.15), int(self.observation_shape[1] * 0.20))
        x=50
        y=50
        
        # Intialise the chopper
        self.chopper = Chopper("chopper", self.x_max, self.x_min, self.y_max, self.y_min)
        self.chopper.set_position(x,y)

        # Intialise the elements 
        self.elements = [self.chopper]

        # Reset the Canvas 
        self.canvas = np.ones(self.observation_shape) * 1

        # Draw elements on the canvas
        self.draw_elements_on_canvas()


        # return the observation
        return self.canvas

    def render(self, mode = "human"):
        assert mode in ["human", "rgb_array"], "Invalid mode, must be either \"human\" or \"rgb_array\""
        if mode == "human":
            cv2.imshow("Game", self.canvas)
            cv2.waitKey(10)

        elif mode == "rgb_array":
            return self.canvas

    def close(self):
        cv2.destroyAllWindows()

    def get_action_meanings(self):
        return {0: "Inc Velocity", 1: "Inc Angle", 2: "Dec Velocity", 3: "Dec Angle", 4: "Inc Velocity and Inc Angle", 5: "Dec Velocity and Dec Angle", 6: "Inc Velocity and Dec Angle", 7: "Dec Velocity and Inc Angle", }

    def has_collided(self, elem1, elem2):
        x_col = False
        y_col = False

        elem1_x, elem1_y = elem1.get_position()
        elem2_x, elem2_y = elem2.get_position()

        if 2 * abs(elem1_x - elem2_x) <= (elem1.icon_w + elem2.icon_w):
            x_col = True

        if 2 * abs(elem1_y - elem2_y) <= (elem1.icon_h + elem2.icon_h):
            y_col = True

        if x_col and y_col:
            return True

        return False

    def target_hit(self, elem1):

        elem1_x, elem1_y = elem1.get_position()
        x=self.dx1
        half=elem1.icon_w//2
        th=elem1_x+half
        r=0

        if(x>=elem1_x and x<=elem1_x+elem1.icon_w):
            if(x>th):
                r=((elem1_x+elem1.icon_w-x)/half)*100
            else:
                r=((x-elem1_x)/half)*100
            print('Lastx: {} | Tarx: {} | Hit Reward: {}'.format(x,elem1_x,r))
            self.miss=0
        else:
            r=-10
            self.miss+=1
        
        

        return r
    
    def rectify(self):
        if(self.v<=0):
            self.v=1
        elif(self.v>=80):
            self.v=79
        if(self.an<=0):
            self.an=0
        elif(self.an>=(90)*(m.pi/180)):
            self.an=(90)*(m.pi/180)
        return 


    
    def step(self, action):
        # Flag that marks the termination of an episode
        done = False

        # Assert that it is a valid action 
        assert self.action_space.contains(action), "Invalid Action"

        # Decrease the fuel counter 
        self.fuel_left -= 1 

        # Reward for executing a step.
        reward = 1
        dx=0
        dy=0
        k=1
        g=9.8
        # Actions
        if action == 0: 
            self.v+=1
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 1:
            self.an+=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        if action == 2: 
            self.v-=1
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 3:
            self.an-=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 4:
            self.v+=1
            self.an+=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 5:
            self.v-=1
            self.an-=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 6:
            self.v+=1
            self.an-=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        elif action == 7:
            self.v-=1
            self.an+=(0.5)*(m.pi/180)
            self.rectify()
            dx = ((self.v*k)*np.cos(self.an))
            dy = ((self.v*k)*np.sin(self.an))-((0.5*g)*(k**2))
        print(self.v,self.an,dx,dy)

        # Spawn a target at the bottom edge with prob 0.01
        if self.miss==0:
            # Spawn a fuel tank
            spawned_target = Target("target_{}".format(self.target_count), self.x_max, 0, 600, 0)
            self.target_count += 1

            # Compute the x,y co-ordinates of the position from where the fuel tank has to be spawned
            # Horizontally, the position is randomly chosen from the list of permissible values and 
            # vertically, the position is on the bottom edge
            target_x = random.randrange(self.x_min, self.x_max)
            #target_x=200
            target_y = self.y_max
            spawned_target.set_position(target_x, target_y)

            # Append the spawned fuel tank to the elemetns currently present in the Env.
            self.elements.append(spawned_target)   


        # For elements in the Ev
        for elem in self.elements:
            if isinstance(elem, Bird):
                # If the bird has reached the left edge, remove it from the Env
                if elem.get_position()[0] <= self.x_min:
                    self.elements.remove(elem)
                else:
                    # Move the bird left by 5 pts.
                    elem.move(-5,0)

                # If the bird has collided.
                if self.has_collided(self.chopper, elem):
                    # Conclude the episode and remove the chopper from the Env.
                    done = True
                    reward = -10
                    self.elements.remove(self.chopper)

            if isinstance(elem, Fuel):
                # If the fuel tank has reached the top, remove it from the Env
                if elem.get_position()[1] <= self.y_min:
                    self.elements.remove(elem)
                else:
                    # Move the Tank up by 5 pts.
                    elem.move(0, -5)

                # If the fuel tank has collided with the chopper.
                if self.has_collided(self.chopper, elem):
                    # Remove the fuel tank from the env.
                    self.elements.remove(elem)

                    # Fill the fuel tank of the chopper to full.
                    self.fuel_left = self.max_fuel

            if isinstance(elem, Target):
                # If the fuel tank has reached the top, remove it from the Env
                if elem.get_position()[1] <= self.y_min:
                    self.elements.remove(elem)
                else:
                    # Move the Tank up by 5 pts.
                    elem.move(0,0)
                    
                if self.has_collided(self.chopper, elem):
                    # Conclude the episode and remove the chopper from the Env.
                    reward = self.target_hit(elem)
                    self.elements.remove(self.chopper)
                    self.miss=0

                # If the fuel tank has collided with the chopper.
                if (self.target_count>=2):
                    # Remove the fuel tank from the env.
                    self.elements.remove(elem)
                    self.target_count-=1
                reward=self.target_hit(elem)
                self.hr+=reward
                if(self.miss>=100):
                    done=True
                            
        for elem in self.elements:
            if isinstance(elem, Target):
                # If the fuel tank has reached the top, remove it from the Env
                if elem.get_position()[1] <= self.y_min:
                    self.elements.remove(elem)
                else:
                    # Move the Tank up by 5 pts.
                    elem.move(0,0)
        
                if self.has_collided(self.chopper, elem):
                    # Conclude the episode and remove the chopper from the Env.
                    reward = self.target_hit(elem)
                    self.elements.remove(self.chopper)
                    hit=True
                    #self.miss=0
                
                    


        # Increment the episodic return
        self.ep_return += 1

        # Draw elements on the canvas
        self.draw_elements_on_canvas()

        # If out of fuel, end the episode.
        if self.fuel_left == 0:
            done = True

        return self.canvas, reward, done, []

In [15]:
class Point(object):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        self.x = 0
        self.y = 0
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.name = name
    
    def set_position(self, x, y):
        self.x = self.clamp(x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(y, self.y_min, self.y_max - self.icon_h)
    
    def get_position(self):
        return (self.x, self.y)
    
    def move(self, del_x, del_y):
        self.x += int(del_x)
        self.y += int(del_y)
        
        self.x = self.clamp(self.x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(self.y, self.y_min, self.y_max - self.icon_h)

    def clamp(self, n, minn, maxn):
        return max(min(maxn, n), minn)

In [16]:
class Chopper(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Chopper, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread("field.jpg") / 255.0
        self.icon_w = 64
        self.icon_h = 64
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))

    
class Bird(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Bird, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread("bird.png") / 255.0
        self.icon_w = 32
        self.icon_h = 32
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))
    
class Fuel(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Fuel, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread("fuel.png") / 255.0
        self.icon_w = 32
        self.icon_h = 32
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))
        
class Target(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Target, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread("target1.jpg") / 255.0
        self.icon_w = 64
        self.icon_h = 64
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))

In [17]:
from IPython import display

env = ChopperScape()
obs = env.reset()


while True:
    # Take a random action
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

    # Render the game
    env.render()

    if done == True:
        break
        
#     try:
#         action = env.action_space.sample()
#         obs, reward, done, info = env.step(action)

#         # Render the game
#         env.render()

#         if done == True:
#             break
#     except:
#         break

env.close()